In [200]:
import re
from collections import deque, defaultdict
from math import prod
from pprint import pp

class Number:
  def __init__(self, monkey, number):
    self.monkey = monkey
    self.original_number = number
    self.number = number
    self.ratios = set()
  def __repr__(self):
    return f'{self.monkey}: {self.original_number}'
  def __hash__(self):
    return hash((self.monkey, self.original_number))
    

sample = """Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1"""


def getdata(data):
    monkeys = map(int, re.findall(r'Monkey (\d+)', data))
    starts = re.findall(r'Starting items: (.*)', data)
    starts = [deque(int(i.strip()) for i in start.split(",") if i.strip())for start in starts]
    operations = re.findall(r'Operation: new = (.*)', data)
    tests = map(int, re.findall(r'Test: divisible by (\d+)', data))
    iftrue = map(int, re.findall(r'If true: throw to monkey (\d+)', data))
    iffalse = map(int, re.findall(r'If false: throw to monkey (\d+)', data))
    return {m: (s,o,t,tu,fa) for m,s,o,t,tu,fa in zip(monkeys, starts, operations, tests, iftrue, iffalse)}


def get_diff_data(data):
    monkeys = map(int, re.findall(r'Monkey (\d+)', data))
    starts = re.findall(r'Starting items: (.*)', data)
    starts = [deque(Number(monkey, int(i.strip())) for i in start.split(",") if i.strip()) for monkey, start in enumerate(starts)]
    operations = re.findall(r'Operation: new = (.*)', data)
    tests = map(int, re.findall(r'Test: divisible by (\d+)', data))
    iftrue = map(int, re.findall(r'If true: throw to monkey (\d+)', data))
    iffalse = map(int, re.findall(r'If false: throw to monkey (\d+)', data))
    return {m: (s,o,t,tu,fa,[0]) for m,s,o,t,tu,fa in zip(monkeys, starts, operations, tests, iftrue, iffalse)}




def do_stuff(monkeys):
    monkey_counter = defaultdict(int)
    for _ in range(20):
        for monkey, (starts, operation, test, iftrue, iffalse) in monkeys.items():
            while starts:
                monkey_counter[monkey] += 1
                old = starts.popleft()
                new = eval(operation)
                new = new // 3
                if not new % test:
                    monkeys[iftrue][0].append(new)
                else:
                    monkeys[iffalse][0].append(new)
                
    print(monkeys)
    print(prod(sorted(monkey_counter.values())[-2:]))


monkeys = getdata(sample)
do_stuff(monkeys)

{0: (deque([10, 12, 14, 26, 34]), 'old * 19', 23, 2, 3), 1: (deque([245, 93, 53, 199, 115]), 'old + 6', 19, 2, 0), 2: (deque([]), 'old * old', 13, 1, 3), 3: (deque([]), 'old + 3', 17, 0, 1)}
10605


In [201]:

monkeys = getdata(open('d11.input').read())
do_stuff(monkeys)


{0: (deque([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 7, 7, 7, 7]), 'old * 7', 11, 5, 6), 1: (deque([3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6]), 'old * 17', 19, 4, 2), 2: (deque([]), 'old + 2', 5, 7, 4), 3: (deque([2, 2, 2, 2, 2, 2, 2]), 'old + 1', 2, 2, 1), 4: (deque([]), 'old + 6', 13, 7, 0), 5: (deque([]), 'old * old', 7, 6, 3), 6: (deque([]), 'old + 3', 3, 1, 3), 7: (deque([]), 'old + 4', 17, 0, 5)}
56350


In [202]:
def do_different_stuff(monkeys, cycles=20, debug=False):
    monkey_chain = defaultdict(list)
    factor = prod(m[2] for m in monkeys.values())
    for c in range(cycles):
        for monkey, (starts, operation, test, iftrue, iffalse, counter) in monkeys.items():
            while starts:
                counter[0] += 1
                number = starts.popleft()
                old = number.number
                new = eval(operation) % factor
                monkey_chain[number].append(monkey)
                number.number = new
                # new = new // 3
                # new = (new // factor) + (new % factor)
                if not new % test:
                    monkeys[iftrue][0].append(number)
                else:
                    monkeys[iffalse][0].append(number)
                
    debug and print({k: v[0] for k,v in monkeys.items()})
    return monkey_chain
    
    # print(prod(sorted(monkey_counter.values())[-2:]))



In [203]:


monkeys = get_diff_data(sample)

pp(monkeys)

monkey_chain = do_different_stuff(monkeys, 10000)  

print(prod(sorted([v[-1][0] for v in monkeys.values()])[-2:] ))



{0: (deque([0: 79, 0: 98]), 'old * 19', 23, 2, 3, [0]),
 1: (deque([1: 54, 1: 65, 1: 75, 1: 74]), 'old + 6', 19, 2, 0, [0]),
 2: (deque([2: 79, 2: 60, 2: 97]), 'old * old', 13, 1, 3, [0]),
 3: (deque([3: 74]), 'old + 3', 17, 0, 1, [0])}
2713310158


In [204]:
monkeys = get_diff_data(open('d11.input').read())

pp(monkeys)
monkey_chain = do_different_stuff(monkeys, 10000)  
print(prod(sorted([v[-1][0] for v in monkeys.values()])[-2:] ))

{0: (deque([0: 97, 0: 81, 0: 57, 0: 57, 0: 91, 0: 61]),
     'old * 7',
     11,
     5,
     6,
     [0]),
 1: (deque([1: 88, 1: 62, 1: 68, 1: 90]), 'old * 17', 19, 4, 2, [0]),
 2: (deque([2: 74, 2: 87]), 'old + 2', 5, 7, 4, [0]),
 3: (deque([3: 53, 3: 81, 3: 60, 3: 87, 3: 90, 3: 99, 3: 75]),
     'old + 1',
     2,
     2,
     1,
     [0]),
 4: (deque([4: 57]), 'old + 6', 13, 7, 0, [0]),
 5: (deque([5: 54, 5: 84, 5: 91, 5: 55, 5: 59, 5: 72, 5: 75, 5: 70]),
     'old * old',
     7,
     6,
     3,
     [0]),
 6: (deque([6: 95, 6: 79, 6: 79, 6: 68, 6: 78]), 'old + 3', 3, 1, 3, [0]),
 7: (deque([7: 61, 7: 97, 7: 67]), 'old + 4', 17, 0, 5, [0])}
13954061248
